# Complete Workflow

This workflow provides a complete working example to develop an unstructured mesh for an integrated hydrologic model based on HUCs.  It is the default workflow for integrated hydrology simulations for Exasheds Simulation Campaign 2.

It uses the following datasets:

* `NHD Plus` for the watershed boundary and hydrography.
* `NED` for elevation
* `NLCD` for land cover/transpiration/rooting depths
* `GLYHMPS` geology data for structural formations
* `SoilGrids 2017` for depth to bedrock and soil texture information
* `SSURGO` for soil data, where available, in the top 2m.

Given some basic inputs (in the next cell) including a NAME, this workflow creates the following files (noting that some suffixes may be appended to the user-provided NAME in homogeneous cases):

* Mesh file: `{NAME}.exo`, includes all labeled sets
* Forcing: DayMet data -- daily raster of precip, RH, incoming radiation, etc.
  - `{NAME}_DayMet_1980_2023.h5`, the DayMet data on this watershed
  - `{NAME}_DayMet_typical_1980_2023.h5`, a "typical year" of DayMet, smoothed for spinup purposes, then looped 40 years
* Forcing: LAI data -- every 4 days, time series by land cover type of LAI.  Note, the raw inputs to this are not done by NAME, but by an (optional, defaults to NAME) MODIS_NAME variable.  Since WW does not currently download MODIS, one might want to use a file of a different name to provide MODIS data.  The times of this MODIS data are hard-coded too -- this is all a bit wonky and will remain so until we get around to adding a file manager for MODIS data.
  - `{NAME}_MODIS_LAI_smoothed_2002_2023.h5`, the LAI, interpolated and smoothed from the raw MODIS data
  - `{NAME}_MODIS_LAI_typical_1980_2023.h5`, a "typical year" of LAI, smoothed for spinup purposes then looped 40 years
* Input files: ATS xml files
  - `spinup-steadystate-{NAME}.xml` the steady-state solution based on uniform application of mean rainfall rate
  - `spinup-cyclic_steadystate-{NAME}.xml` the cyclic steady state based on typical years
  - `transient-{NAME}.xml` the forward model


In [ ]:
# these can be turned on for development work
%load_ext autoreload
%autoreload 2
import numpy as np

In [ ]:
name = 'Naches' # name the domain, used in filenames, etc
hucs = ['17030002'] # a list of HUCs to run
def get_huc12(hucs):
    huc12_list = []
    for huc in hucs:
        if len(huc) == 12:
            huc12_list.append(huc)
        elif len(huc) == 10:
            for i in range(1,20):
                huc12_list.append(huc+str(i).zfill(2))
        elif len(huc) == 8:
            for i in range(1,20):
                for j in range(1,20):
                    huc12_list.append(huc+str(i).zfill(2)+str(j).zfill(2))
        else:
            print('need huc8, huc10 or huc12')
    return huc12_list
hucs = get_huc12(hucs)
print(hucs[:10])

In [ ]:
fire = True
if fire:
    bs_ptcloud_file_path = f'../data-processed/{name}/fire/xyz.xyz'
    perimeter_file_path = f'../data-processed/{name}/fire/perimeter.i2s'
    postfire_perm_file_path = f'../data-processed/{name}/fire/postfire_perm.txt'
    postfire_perm_4d_file_path = f'../data-processed/{name}/fire/post.xyz'

In [ ]:
# Parameters cell -- this provides all parameters that can be changed via pipelining to generate a new watershed.
huc_level = 12 # if provided, an int setting the level at which to include HUC boundaries

# geometric parameters
simplify_hucs = 100 # length scale to target average edge
simplify_rivers = 100
stream_outlet_width = 300 # half-width to track a labeled set on which to get discharge
ignore_small_rivers = 2 # ignore rivers which have this or fewer reaches.  likely they are irrigation ditches
                        # or other small features which make things complicated but likely don't add much value
prune_by_area_fraction = 0.01 # ignore reaches whose accumulated catchment area is less than this fraction of the
                              # full domain's area
prune_by_area_fraction_waterbodies = None
num_smoothing_sweeps = 5 # number of times to smooth the DEM prior to elevating

# simulation control
start_year = 1980  # year to start and end simulation simulation -- note these start and end Oct 1 of the year
end_year = 2023
min_porosity = 0.05 # minimum porosity considered too small
max_permeability = 1.e-10 # max value allowed for permeability
max_vg_alpha = 1.e-3 # max value of van Genuchten's alpha -- our correlation is not valid for some soils

# triangle refinement control
include_rivers = True
# refine_d0 = 100
# refine_d1 = 500
# refine_A0 = 8000
# refine_A1 = 50000
meshsize = 250
factor = 4
refine_d0 = meshsize*3
refine_A0 = meshsize**2/2
refine_d1 = meshsize*30
refine_A1 = (np.round(meshsize*factor))**2/2

# soil structure
use_geologic_layer = True

# logistics
generate_plots = True # plots take time to make and aren't always needed
generate_daymet = True # potentially don't do Met data forcing
generate_modis = False

include_heterogeneous = True
include_homogeneous = False # if true, also write files for homogeneous runs
include_homogeneous_wrm = False # if true, also write files for homogeneous WRMs
include_homogeneous_wrm_porosity = False # if true, also write files for homogeneous porosity and WRMs
include_homogeneous_wrm_permeability = False # if true, also write files for homogeneous perm and WRMs

log_to_file = False  # if true, write to file instead of in the notebook output
figsize = (6,6)
figsize_3d = (8,6)

In [ ]:
# parameter checking
# assert(simplify_hucs > 0 and simplify_hucs < 300)
assert(ignore_small_rivers == None or (ignore_small_rivers >= 0 and ignore_small_rivers <= 100))
assert(prune_by_area_fraction == None or (prune_by_area_fraction >= 0 and prune_by_area_fraction < 1))
# assert(start_year >= 1980 and start_year < 2023)

if type(hucs) is str:
    assert(hucs[0] == '[')
    assert(hucs[-1] == ']')
    hucs = hucs[1:-1]
    hucs = hucs.split(',')
    hucs = [h.strip() for h in hucs]
    if hucs[-1] == '':
        hucs = hucs[:-1]

if huc_level is None:
    huc_level = len(hucs[0])
else:
    assert(huc_level >= len(hucs[0]))
huc_key = f'HUC{huc_level}'

if prune_by_area_fraction_waterbodies is None:
    prune_by_area_fraction_waterbodies = prune_by_area_fraction * 0.1

In [ ]:
# a dictionary of outputs -- will include all filenames generated
outputs = {}

In [ ]:
# conda package imports
import os,sys
import numpy as np
import shapely
from shapely.geometry import mapping
import fiona
import logging
import scipy.ndimage
from matplotlib import pyplot as plt
plt.rcParams['figure.dpi'] = 150
import h5py
import pandas
pandas.options.display.max_columns = None
pandas.options.display.max_rows = 20

# Watershed Workflow
import watershed_workflow
import watershed_workflow.source_list
import watershed_workflow.ui
import watershed_workflow.colors
import watershed_workflow.condition
import watershed_workflow.mesh
import watershed_workflow.split_hucs
import watershed_workflow.soil_properties
import watershed_workflow.daymet
import watershed_workflow.utils

import watershed_workflow.regions

if log_to_file:
    outputs['logfile'] = f'{name}.log'
    # is this right?  the file handle will become stale... test once this is pipelined.
    with open(outputs['logfile'], 'w') as fid:
        watershed_workflow.ui.setup_logging(1,fid)
else:
    watershed_workflow.ui.setup_logging(1,None)

# ats_input_spec library, to be moved to amanzi_xml
import ats_input_spec
import ats_input_spec.public
import ats_input_spec.io

# amanzi_xml, included in AMANZI_SRC_DIR/tools/amanzi_xml
import amanzi_xml.utils.io as aio
import amanzi_xml.utils.search as asearch
import amanzi_xml.utils.errors as aerrors

# from $ATS_SRC_DIR/tools/utils
#import smooth_met_box


In [ ]:
# Note that, by default, we tend to work in the DayMet CRS because this allows us to avoid
# reprojecting meteorological forcing datasets.
crs = watershed_workflow.crs.daymet_crs()
crs

## Sources and setup

Next we set up the source watershed and coordinate system and all data sources for our mesh.  We will use the CRS that is included in the shapefile.

In [ ]:
logging.info("")
logging.info(f"Meshing shape: {hucs}")
logging.info("="*30)


A wide range of data sources are available; here we use the defaults except for using NHD Plus for watershed boundaries and hydrography (the default is NHD, which is lower resolution and therefore smaller download sizes).

In [ ]:
# set up a dictionary of source objects
sources = watershed_workflow.source_list.get_default_sources()
sources['hydrography'] = watershed_workflow.source_list.hydrography_sources['NHD Plus']
sources['HUC'] = watershed_workflow.source_list.huc_sources['NHD Plus']
# sources['DEM'] = watershed_workflow.source_list.dem_sources['NED 1/3 arc-second']
sources['geologic structure'] = watershed_workflow.source_list.FileManagerGLHYMPS('/global/cfs/cdirs/m1800/zhi/ww/scripts/data/soil_structure/GLHYMPS/GLHYMPS.shp')
sources['depth to bedrock'] = watershed_workflow.source_list.FileManagerRaster('/global/cfs/cdirs/m1800/zhi/ww/scripts/data/soil_structure/SoilGrids2017/BDTICM_M_250m_ll.tif')
watershed_workflow.source_list.log_sources(sources)

In [ ]:
# load the huc
my_hucs = []
for huc in hucs:
    _, ws = watershed_workflow.get_hucs(sources['HUC'], huc, huc_level, crs)
    my_hucs.extend(ws)

watershed = watershed_workflow.split_hucs.SplitHUCs(my_hucs)

In [ ]:

### save watershed boundary shapefile
try:
    os.mkdir(f'../data-processed/{name}')
except FileExistsError:
    pass
try:
    os.mkdir(f'../images/{name}')
except FileExistsError:
    pass
outputs['watershed_shapefile_filename'] = f'../data-processed/{name}/{name}_bounds.shp'
# Define a polygon feature geometry with one attribute
schema = {
    'geometry': 'Polygon',
    'properties': {'id': 'int'},
}

# Write a new Shapefile
with fiona.open(outputs['watershed_shapefile_filename'], 'w', 'ESRI Shapefile', schema, crs=watershed_workflow.crs.to_fiona(crs)) as c:
    ## If there are multiple geometries, put the "for" loop here
    c.write({
        'geometry': mapping(watershed.exterior()),
        'properties': {'id': 123},
    })

## Generate the surface mesh

First we'll generate the flattened, 2D triangulation, which builds on hydrography data.  Then we download a digital elevation map from the National Elevation Dataset, and extrude that 2D triangulation to a 3D surface mesh based on interpolation between pixels of the DEM.

### Get river network

This will download the river network from the NHD Plus database, and simplify the network, constructing a tree-like data structure.

In [ ]:
if include_rivers:  
    # download/collect the river network within that shape's bounds
    _, reaches = watershed_workflow.get_reaches(sources['hydrography'], huc, 
                                                watershed.exterior(), crs, crs,
                                                in_network=True, properties=True)
    
    rivers = watershed_workflow.construct_rivers(reaches, method='hydroseq',
                                                 ignore_small_rivers=ignore_small_rivers,
                                                 prune_by_area=prune_by_area_fraction * watershed.exterior().area * 1.e-6,
                                                 remove_diversions=True,
                                                 remove_braided_divergences=True)
else:
    reaches = []
    rivers = []

In [ ]:
# what about the reservoir!
_, waterbodies = watershed_workflow.get_waterbodies(sources['hydrography'], huc, watershed.exterior(), crs, crs, 
                                                    prune_by_area=prune_by_area_fraction_waterbodies*watershed.exterior().area)


In [ ]:
if generate_plots:
    fig, ax = watershed_workflow.plot.get_ax(crs)
    watershed_workflow.plot.hucs(watershed, crs, 'k', ax)
    watershed_workflow.plot.rivers(rivers, crs, 'b', ax)
    watershed_workflow.plot.shplys(waterbodies, crs, 'g', ax, facecolor='c')
    plt.show()

In [ ]:
# # simplify the geometry for a "nicer" discrete object
# watershed_workflow.simplify(watershed, rivers, waterbodies, simplify_hucs=simplify_hucs, simplify_rivers=simplify_rivers,
#                             snap=False, cut_intersections=False)

In [ ]:
tol = 1e-3
import copy
# keeping the originals
rivers_orig=[river.deepcopy() for river in rivers]
watershed_orig=copy.deepcopy(watershed) 

# simplifying 
rivers = watershed_workflow.simplify(watershed, rivers, 
                                     simplify_hucs=simplify_hucs, 
                                     simplify_rivers=simplify_rivers, 
                                     prune_tol=tol, merge_tol=tol, snap_tol=tol,
                                     cut_intersections=False)

# for plotting purpose only
rivers_simplified=[river.deepcopy() for river in rivers] 
watershed_simplified=copy.deepcopy(watershed) 

print('number of reaches in original', len(rivers_orig[0]), 'number of reaches in simplified', len(rivers[0]))

### Generate meshes using river network and watershed shape

Triangulation refinement: refine triangles if their area (in m^2) is greater than A(d), where d is the 
distance from the triangle centroid to the nearest stream.  A(d) is a piecewise linear function -- A = A0 if d <= d0, A = A1 if d >= d1, and linearly interpolates between the two endpoints.

In [ ]:
d0 = refine_d0; d1 = refine_d1
A0 = refine_A0; A1 = refine_A1 # [100, 310]m

# Refine triangles if they get too acute
min_angle = 32 # degrees

# make 2D mesh
diagnostics = True
if generate_plots and diagnostics:
    # mesh_points2, mesh_tris, areas, distances = watershed_workflow.triangulate(watershed, rivers, 
    #                                                refine_distance=[d0,A0,d1,A1],
    #                                                refine_min_angle=min_angle,
    #                                                diagnostics=True)
    mesh_points2, mesh_tris, areas, distances, river_idx = watershed_workflow.triangulate(watershed, rivers,                                                 
                                                   refine_distance=[d0,A0,d1,A1],
                                                   refine_min_angle=min_angle,
                                                   # enforce_delaunay=True,
                                                   diagnostics=True,
                                                   river_region_dist=meshsize)
else:
    mesh_points2, mesh_tris = watershed_workflow.triangulate(watershed, rivers, 
                                                   refine_distance=[d0,A0,d1,A1],
                                                   refine_min_angle=min_angle,
                                                   diagnostics=False)
print('total area=', np.sum(areas), 'm^2')

In [ ]:
river_idx

In [ ]:
str(len(river_idx[river_idx])/len(river_idx)*100)+'%'

### Map mesh to DEM

Download a DEM from USGS NED and elevate the triangle nodes to the DEM.

In [ ]:
# download the needed rasters
dem_profile, dem = watershed_workflow.get_raster_on_shape(sources['DEM'], watershed.exterior(), crs)

In [ ]:
# noting that the DEM is a 30m raster, and we want to run at a coarser resolution of ~100-300m, 
# the DEM will look quite rough.  Smooth a small amount.  Note better algorithms could be used 
# here, but for now we just use Gaussian smoothing.
if num_smoothing_sweeps > 0:
    dem_sm = scipy.ndimage.gaussian_filter(dem, num_smoothing_sweeps, mode='nearest')
else:
    dem_sm = dem[:]

if generate_plots:
    fig, axs = plt.subplots(1,2, figsize=(10,5))
    axs[0].imshow(dem)
    axs[0].set_title('unsmoothed')
    axs[1].imshow(dem_sm)
    txt = axs[1].set_title('smoothed')

In [ ]:
# elevate the x,y points onto the DEM to get a z coordinate
mesh_points3 = watershed_workflow.elevate(mesh_points2, crs, dem_sm, dem_profile)

In [ ]:
# construct the 2D mesh
m2 = watershed_workflow.mesh.Mesh2D(mesh_points3.copy(), list(mesh_tris), crs=crs)

In [ ]:
# hydrologically condition the mesh, removing pits.
#
# replace conditioned mesh where there are water bodies!
all_bodies = shapely.geometry.MultiPolygon(waterbodies)

waterbody_mask = np.zeros((len(m2.conn),), 'i')
for i,p in enumerate(m2.centroids):
    if all_bodies.contains(shapely.geometry.Point(p)):
        waterbody_mask[i] = 1
        
print(waterbody_mask.shape)
print(waterbody_mask.sum())
# NOTE: this fills reservoirs as well!  Might have to think about how to allow some pits!
# watershed_workflow.condition.fill_pits_dual(m2, is_waterbody=waterbody_mask)

In [ ]:
# image the diff of the elevation
if generate_plots:
    diff = m2.coords[:,2] - mesh_points3[:,2]
    
    fig, ax = watershed_workflow.plot.get_ax(crs, figsize=(12,10))

    mp = watershed_workflow.plot.triangulation(m2.coords, m2.conn, crs, ax=ax, 
                                 color=diff, edgecolor='white', linewidth=0.2)

In [ ]:
# identify outlets by the elevation map
#watershed_workflow.split_hucs.find_outlets_by_elevation(watershed, crs, dem_sm, dem_profile)

print(len(rivers))
rivers = sorted(rivers, key=len)
watershed_workflow.split_hucs.find_outlets_by_hydroseq(watershed, rivers[-1])

if generate_plots:
    fig, ax = watershed_workflow.plot.get_ax(crs, figsize=(12,10))
    colors = watershed_workflow.colors.enumerated_colors(len(watershed), palette=4)
    watershed_workflow.plot.hucs(watershed, crs, ax=ax, color=colors, linewidth=1, facecolor='color', alpha=0.4)
    watershed_workflow.plot.rivers(rivers, crs, ax=ax, colors='b', linewidth=0.5)

In [ ]:
if generate_plots:
    # plot the resulting surface mesh
    fig, ax = watershed_workflow.plot.get_ax(crs, window=[0.05,0.1,0.9,0.8], figsize=(12,10))
    cbax = fig.add_axes([0.05,0.05,0.9,0.05])
    
    mp = watershed_workflow.plot.triangulation(m2.coords, m2.conn, crs, ax=ax, 
                                 color='elevation', edgecolor='white', linewidth=0.2)
    cbar = fig.colorbar(mp, orientation="horizontal", cax=cbax)
    watershed_workflow.plot.hucs(watershed, crs, ax=ax, color='k', linewidth=1)
    watershed_workflow.plot.rivers(rivers, crs, ax=ax, color='red', linewidth=1)
    ax.set_aspect('equal', 'datalim')
    txt = cbar.ax.set_xlabel('elevation [m]', loc='center')
    fig.savefig(f'../images/{name}/{name}_dem')

In [ ]:
# # add labeled sets for subcatchments and outlets
# watershed_workflow.mesh.add_watershed_regions_and_outlets(m2, watershed, stream_outlet_width, 
#                                                           labels=[p.properties[huc_key] for p in watershed.polygons()])


In [ ]:
# add labeled sets for subcatchments and outlets
watershed_workflow.regions.add_watershed_regions_and_outlets(m2, watershed, outlet_width=stream_outlet_width, 
                                                          labels=[p.properties[huc_key] for p in watershed.polygons()], exterior_outlet= True)

# add labeled sets for river corridor cells
# watershed_workflow.regions.add_river_corridor_regions(m2, rivers)

In [ ]:
for ls in m2.labeled_sets:
    print(f'{ls.setid} : {ls.entity} : {len(ls.ent_ids)} : "{ls.name}"')

## Surface properties

Meshes interact with data to provide forcing, parameters, and more in the actual simulation.  Specifically, we need vegetation type on the surface to provide information about transpiration and subsurface structure to provide information about water retention curves, etc.

We'll start by downloading and collecting land cover from the NLCD dataset, and generate sets for each land cover type that cover the surface.  Likely these will be some combination of grass, deciduous forest, coniferous forest, and mixed forest.

In [ ]:
# download the NLCD raster
lc_profile, lc_raster = watershed_workflow.get_raster_on_shape(sources['land cover'], 
                                                     watershed.exterior(), crs)

# resample the raster to the triangles
lc = watershed_workflow.values_from_raster(m2.centroids, crs, lc_raster, lc_profile)

# what land cover types did we get?
logging.info('Found land cover dtypes: {}'.format(lc.dtype))
logging.info('Found land cover types: {}'.format(set(lc)))


In [ ]:

### land cover histogram
u, count = np.unique(lc, return_counts=True)
count_sort_ind = np.argsort(-count)
lc_dict = dict(zip(u[count_sort_ind], count[count_sort_ind]))
print(lc_dict)
aa = count[count_sort_ind]
print(aa[0]/np.sum(aa)*100, aa[1]/np.sum(aa)*100, aa[2]/np.sum(aa)*100, aa[3]/np.sum(aa)*100)
plt.hist(lc, bins='auto')
plt.ylabel('counts')
plt.xlabel('index')
plt.show()

In [ ]:
if generate_plots:
    # plot the image
    # -- get the NLCD colormap which uses official NLCD colors and labels
    nlcd_indices, nlcd_cmap, nlcd_norm, nlcd_ticks, nlcd_labels = \
                watershed_workflow.colors.generate_nlcd_colormap(lc)

    fig = plt.figure(figsize=figsize)
    ax = watershed_workflow.plot.get_ax(crs, fig)
    polys = watershed_workflow.plot.mesh(m2, crs, ax=ax, color=lc, cmap=nlcd_cmap, norm=nlcd_norm, edgecolor='none', 
                                     facecolor='color', linewidth=0.5)

    # nlcd_labels.pop()
    watershed_workflow.colors.colorbar_index(ncolors=len(nlcd_indices), cmap=nlcd_cmap, labels = nlcd_labels) 

    ax.set_title("NLCD land cover index")
    ext = ax.axis('off')
    fig.savefig(f'../images/{name}/{name}_land_cover_raw')

In [ ]:
nlcd_indices

In [ ]:
nlcd_labels

In [ ]:
len(nlcd_indices), len(nlcd_labels)

In [ ]:
nlcd_cmap

In [ ]:
type(nlcd_cmap)

In [ ]:
nlcd_labels_dict = dict(zip(nlcd_indices, nlcd_labels))
nlcd_labels_dict

In [ ]:
# we don't really need all of these.  Keep Evergreen, Deciduous, Shrub, and merge the rest into "Other"
nlcd_color_new = 99 * np.ones_like(lc)

groupings = {
    42 : ['Evergreen Forest',],
    # 41 : ['Deciduous Forest', 'Mixed Forest', 'Woody Wetlands'],
    52 : ['Dwarf Scrub', 'Shrub/Scrub', 'Grassland/Herbaceous', 'Sedge/Herbaceous', 
                     'Pasture/Hay', 'Cultivated Crops'],
    71 : ['Grassland/Herbaceous'],
}

for k,v in groupings.items():
    for label in v:
        index = sources['land cover'].indices[label]
        nlcd_color_new[np.where(lc == index)] = k
    
print(nlcd_color_new)

In [ ]:

### land cover histogram
u, count = np.unique(nlcd_color_new, return_counts=True)
count_sort_ind = np.argsort(-count)
print(u[count_sort_ind])
print(count[count_sort_ind])
aa = count[count_sort_ind]
print(aa[0]/np.sum(aa)*100, aa[1]/np.sum(aa)*100, aa[2]/np.sum(aa)*100)#, aa[3]/np.sum(aa)*100)
plt.hist(nlcd_color_new, bins='auto')
plt.ylabel('counts')
plt.xlabel('index')
plt.show()

In [ ]:
# plot the updated image, adding "other"
nlcd_color_new_other_as_water = np.where(nlcd_color_new == 99, 11, nlcd_color_new)

# -- get the NLCD colormap which uses official NLCD colors and labels
nlcd_indices, nlcd_cmap, nlcd_norm, nlcd_ticks, nlcd_labels = \
            watershed_workflow.colors.generate_nlcd_colormap(nlcd_color_new_other_as_water)

# make (water, 11) into (other, 99)
nlcd_labels[0] = 'Other'
nlcd_indices[0] = 99

if generate_plots:
    fig, ax = watershed_workflow.plot.get_ax(crs)


    polys = watershed_workflow.plot.mesh(m2, crs, ax=ax, color=nlcd_color_new_other_as_water, 
                           cmap=nlcd_cmap, norm=nlcd_norm, edgecolor='none', 
                           facecolor='color', linewidth=0.5)
    # nlcd_labels.pop()
    watershed_workflow.colors.colorbar_index(ncolors=len(np.unique(nlcd_color_new_other_as_water)), 
                               cmap=nlcd_cmap, labels = nlcd_labels) 

    ax.set_title("Land Cover Index")
    ext = ax.axis('off')
    fig.savefig(f'../images/{name}/{name}_land_cover')



In [ ]:
# # add labeled sets to the mesh for NLCD
# nlcd_labels_dict = dict(zip(nlcd_indices, nlcd_labels))
# watershed_workflow.mesh.add_nlcd_labeled_sets(m2, nlcd_color_new, nlcd_labels_dict)

In [ ]:
# add labeled sets to the mesh for NLCD
nlcd_labels_dict = dict(zip(nlcd_indices, nlcd_labels))
watershed_workflow.regions.add_nlcd_labeled_sets(m2, nlcd_color_new, nlcd_labels_dict)

In [ ]:

### land cover histogram
print(nlcd_labels_dict)

In [ ]:
for ls in m2.labeled_sets:
    print(f'{ls.setid} : {ls.entity} : {len(ls.ent_ids)} : "{ls.name}"')

## Subsurface properties

The default model uses GLHYMPS to identify geologic formations, and 

In [ ]:
# download the NRCS soils data as shapes and project it onto the mesh
#
soil_profile, soil_survey, soil_survey_props = \
        watershed_workflow.get_shapes(sources['soil structure'], list(watershed.polygons()), crs, 
                                                     crs, properties=True)

# -- determine the NRCS mukey for each soil unit; this uniquely identifies soil 
#    properties
soil_ids = list(soil_survey_props['mukey'][:])
soil_survey_props.set_index('mukey', inplace=True)

# -- color a raster by the polygons (this makes identifying a triangle's value much 
#    more efficient)
soil_color_profile, soil_color_raster = watershed_workflow.color_raster_from_shapes(soil_survey, crs, soil_ids,
                                                                                    watershed.exterior().bounds, 10, crs, -1)

# -- resample the raster to the triangles
soil_color = watershed_workflow.values_from_raster(m2.centroids, crs, 
                                         soil_color_raster, soil_color_profile)


In [ ]:
soil_survey_props


In [ ]:
print('soil_color: ', soil_color)
print('soil_color.shape: ', soil_color.shape)
print('len(np.unique(soil_ids)): ', len(np.unique(soil_ids)))
print('len(np.unique(soil_color)): ', len(np.unique(soil_color)))
print('np.unique(soil_ids): ', np.unique(soil_ids))
print('np.unique(soil_color): ', np.unique(soil_color))

In [ ]:
if generate_plots:
    # plot the soil mukey
    indices, cmap, norm, ticks, labels = watershed_workflow.colors.generate_indexed_colormap(soil_color, cmap='tab20c')
    fig, ax = watershed_workflow.plot.get_ax(crs)

    mp = watershed_workflow.plot.mesh(m2, crs, ax=ax, facecolor='color',
                            linewidth=0, color=soil_color, 
                            cmap=cmap, norm = norm
                           )

    # watershed_workflow.colors.colorbar_index(ncolors=len(np.unique(soil_color)), cmap=cmap, labels=labels) 

    ax.set_title('soil type index')
    ax.axis('off')


In [ ]:
# what does soil thickness look like?
soil_thickness = np.nan * np.ones(soil_color.shape, 'd')
for mukey in soil_survey_props.index:
    soil_thickness[soil_color == mukey] = soil_survey_props.loc[mukey,'thickness [cm]']

soil_thickness = soil_thickness / 100
soil_thickness = np.where(np.isnan(soil_thickness), 2.0, soil_thickness)

if generate_plots:
    fig, ax = watershed_workflow.plot.get_ax(crs)
    mp = watershed_workflow.plot.triangulation(mesh_points3, mesh_tris, crs, ax=ax, 
                                 color=soil_thickness, edgecolor='none', cmap='jet')
    ax.set_title('soil thickness [m]')
    cb = fig.colorbar(mp, fraction=0.04, pad=0.04)
    ax.axis('off')

print('Median soil thickness [m] = ', np.median(soil_thickness))



In [ ]:
if generate_plots:
    # plot of porosity from SSURGO
    iprop = np.empty(soil_color.shape, 'd')
    for mukey in soil_survey_props.index:
        iprop[soil_color == mukey] = soil_survey_props.loc[ mukey,'porosity [-]']

    fig, ax = watershed_workflow.plot.get_ax(crs)
    mp = watershed_workflow.plot.triangulation(mesh_points3, mesh_tris, crs, ax=ax, 
                                 color=iprop, edgecolor='none', cmap='jet')
    ax.set_title('soil porosity [-]')
    cb = fig.colorbar(mp, fraction=0.04, pad=0.04, extend="both", shrink=0.6)
    ax.axis('off')

    print('Median porosity [-] = ', np.nanmedian(iprop))
    fig.savefig(f'../images/{name}/{name}_soil_porosity')

In [ ]:
if generate_plots:
    # plot of permeability
    iprop = np.empty(soil_color.shape, 'd')
    for mukey in soil_survey_props.index:
        iprop[soil_color == mukey] = soil_survey_props.loc[ mukey,'permeability [m^2]']

    fig, ax = watershed_workflow.plot.get_ax(crs)
    mp = watershed_workflow.plot.triangulation(mesh_points3, mesh_tris, crs, ax=ax, 
                                 color=np.log10(iprop), edgecolor='none', cmap='jet')
    ax.set_title('soil permeability [m^2]')
    cb = fig.colorbar(mp, fraction=0.04, pad=0.04, extend="both", shrink=0.6)
    cb.ax.set_title('log K')
    ax.axis('off')

    print('Min k [m^2] = ', np.nanmin(iprop))
    print('Max k [m^2] = ', np.nanmax(iprop))
    print('Mean k [m^2] = ', np.nanmean(iprop))
    print('Median k [m^2] = ', np.nanmedian(iprop))
    fig.savefig(f'../images/{name}/{name}_soil_permeability')

In [ ]:
print('# of nan: ', len(iprop[np.isnan(iprop)]))
for i,j in enumerate(iprop):
    if np.isnan(j):
        iprop[i] = np.nanmean(iprop)
print('# of nan: ', len(iprop[np.isnan(iprop)]))

In [ ]:
if generate_plots:
    # plot of permeability

    fig, ax = watershed_workflow.plot.get_ax(crs)
    mp = watershed_workflow.plot.triangulation(mesh_points3, mesh_tris, crs, ax=ax, 
                                 color=np.log10(iprop), edgecolor='none', cmap='jet')
    ax.set_title('nan removed\nsoil permeability [m^2]')
    cb = fig.colorbar(mp, fraction=0.04, pad=0.04, extend="both", shrink=0.6)
    cb.ax.set_title('log K')
    ax.axis('off')

    print('Min k [m^2] = ', np.nanmin(iprop))
    print('Max k [m^2] = ', np.nanmax(iprop))
    print('Mean k [m^2] = ', np.nanmean(iprop))
    print('Median k [m^2] = ', np.nanmedian(iprop))
    fig.savefig(f'../images/{name}/{name}_soil_permeability')

In [ ]:
if fire:
    m2 = watershed_workflow.mesh.Mesh2D(mesh_points3.copy(), list(mesh_tris), crs=crs)
    xyz = m2.centroids
(xyz, xyz.shape) if fire else fire

In [ ]:
if fire:
    from shapely.geometry import Point, Polygon
    d = np.loadtxt(bs_ptcloud_file_path)
    x, y, z = d[:,0], d[:,1], d[:,2]
(d, d.shape) if fire else fire

In [ ]:
if fire:
    print('before clean =', np.unique(z))
    for i in [0,5,6,7]:
        x, y, z = x[z!=i], y[z!=i], z[z!=i]
    print('after clean =', np.unique(z))

In [ ]:
if fire:
    burn_severity_nodes = [(x[i], y[i]) for i in range(len(x))]
    polygon = Polygon(burn_severity_nodes)
# polygon if fire else fire

In [ ]:
if fire:
    perimeter = np.loadtxt(perimeter_file_path)
    px, py = perimeter[:,0], perimeter[:,1]
(perimeter, perimeter.shape) if fire else fire

In [ ]:
if fire:
    fire_perimeter_nodes = [(px[i], py[i]) for i in range(len(px))]
    fireperimeter = Polygon(fire_perimeter_nodes)
    
fireperimeter if fire else fire

In [ ]:
if fire:
    fig, ax1 = plt.subplots(1, 1, figsize=(5,5))
    diff = np.ones(np.shape(m2.coords[:,2]))
    watershed_workflow.plot.triangulation(m2.coords, m2.conn, crs, ax=ax1, color=diff, edgecolor='none', linewidth=0.2, cmap='Blues', vmin=0, vmax=2)
    p1 = ax1.scatter(x[z==1], y[z==1], c='white', s=0.001)
    p1 = ax1.scatter(x[z==2], y[z==2], c='wheat', s=0.001)
    p1 = ax1.scatter(x[z==3], y[z==3], c='salmon', s=0.001)
    p1 = ax1.scatter(x[z==4], y[z==4], c='firebrick', s=0.001)
    ax1.plot(px, py, 'k')
    # ax1.set_title('burn severity - point cloud')
    ax1.set_title('permeability [m^2]')
    ax1.set_aspect('equal')
    ax1.axis('off')
    # cb = fig.colorbar(p1, fraction=0.04, pad=0.04, extend="both", shrink=0.4)
    cb.ax.set_title('burn\nseverity\n')
    # plt.text(np.mean(px), np.mean(py)*1.016, 'fire perimeter', fontsize=14)
    # plt.text(np.mean(px)/0.99, np.mean(py)*1.016, 'watershed', fontsize=14)
    plt.tight_layout()
    plt.show()

In [ ]:
# if fire:
#     _iprop = np.copy(iprop)
#     for i in range(len(xyz)):
#         _iprop[i] = iprop[i]*0.2 if Point(xyz[i,0], xyz[i,1]).within(fireperimeter) else iprop[i]
        
#     ratio = (iprop-_iprop)/iprop/0.2

#     fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(9,6))
    
#     p1 = ax1.scatter(xyz[:,0], xyz[:,1], c=np.log10(iprop), cmap='PuOr', vmin=-13.25, vmax=-11.25, s=0.005)
#     watershed_workflow.plot.hucs(watershed_workflow.split_hucs.SplitHUCs(my_hucs), crs, ax=ax1, color='navy', lw=0.5)
#     ax1.plot(px, py, 'firebrick', lw=1)
#     ax1.set_title('Pre-fire topsoil\npermeability [m$^2$]')
#     ax1.set_aspect('equal')
#     ax1.axis('off')
#     cb = fig.colorbar(p1, fraction=0.04, pad=0.04, extend="both", shrink=0.25, ax=ax1)
#     cb.ax.set_title('log K\n')
    
#     p2 = ax2.scatter(xyz[:,0], xyz[:,1], c=np.log10(_iprop), cmap='PuOr', vmin=-13.25, vmax=-11.25, s=0.005)
#     watershed_workflow.plot.hucs(watershed_workflow.split_hucs.SplitHUCs(my_hucs), crs, ax=ax2, color='navy', lw=0.5)
#     ax2.plot(px, py, 'firebrick', lw=1)
#     ax2.set_title('Post-fire topsoil\npermeability [m$^2$]')
#     ax2.set_aspect('equal')
#     ax2.axis('off')
#     cb = fig.colorbar(p2, fraction=0.04, pad=0.04, extend="both", shrink=0.25, ax=ax2)
#     cb.ax.set_title('log K\n')
    
#     p3 = ax3.scatter(xyz[:,0][ratio==1], xyz[:,1][ratio==1], c='green', s=0.005, label='no-low')
#     p3 = ax3.scatter(xyz[:,0][ratio==2], xyz[:,1][ratio==2], c='cyan', s=0.005, label='low')
#     p3 = ax3.scatter(xyz[:,0][ratio==3], xyz[:,1][ratio==3], c='yellow', s=0.005, label='moderate')
#     p3 = ax3.scatter(xyz[:,0][ratio==4], xyz[:,1][ratio==4], c='red', s=0.005, label='high')
#     p3 = ax3.scatter(xyz[:,0][ratio==0], xyz[:,1][ratio==0], c='white', s=0.005, label='_nolegend_')
#     watershed_workflow.plot.hucs(watershed_workflow.split_hucs.SplitHUCs(my_hucs), crs, ax=ax3, color='navy', lw=0.5)
#     ax3.plot(px, py, 'firebrick', lw=1)
#     ax3.set_title('(1-Post-fire/Pre-fire)*0.2')
#     ax3.set_aspect('equal')
#     ax3.axis('off')

#     plt.tight_layout()
#     plt.show()

In [ ]:
if fire:
    from scipy.spatial import distance

    def closest_burn_severity(node):
        closest_index = distance.cdist([node], burn_severity_nodes).argmin()
        return z[closest_index]
        # return burn_severity_nodes[closest_index]

    closest_burn_severity((xyz[1234,0], xyz[1234,1]))

In [ ]:
if fire:
    li = []
    for i in range(len(xyz)):
        if Point(xyz[i,0], xyz[i,1]).within(fireperimeter):
            li.append(i)
    print(len(xyz))
    print(len(li))
    print(soil_color[li])
    print(np.unique(soil_color[li]))
    print(len(np.unique(soil_color[li])))

In [ ]:
if fire:
    # recompute = True
    recompute = True
    if recompute:
        from tqdm import trange
        logging.info('start searching closest burn severity')
        _iprop = np.copy(iprop)
        for i in trange(len(xyz)):
            if Point(xyz[i,0], xyz[i,1]).within(fireperimeter):
                 _iprop[i] = iprop[i]*(1-closest_burn_severity((xyz[i,0], xyz[i,1]))*0.2)
        logging.info('end searching closest burn severity')
        np.savetxt(postfire_perm_file_path, _iprop)
    else:
        _iprop = np.loadtxt(postfire_perm_file_path)

In [ ]:
if fire:
    ratio = np.array(list(int(i) for i in (iprop-_iprop)/iprop/0.2))

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(9,6))
    
    p1 = ax1.scatter(xyz[:,0], xyz[:,1], c=np.log10(iprop), cmap='PuOr', vmin=-12.75, vmax=-11, s=0.005)
    watershed_workflow.plot.hucs(watershed_workflow.split_hucs.SplitHUCs(my_hucs), crs, ax=ax1, color='navy', lw=0.5)
    ax1.plot(px, py, 'firebrick', lw=1)
    ax1.set_title('Pre-fire topsoil\npermeability')
    ax1.set_aspect('equal')
    ax1.axis('off')
    cb = fig.colorbar(p1, fraction=0.04, pad=0.04, extend="both", shrink=0.25, ax=ax1)
    cb.ax.set_title('log K\n[m$^2$]', fontsize=10)
    
    p2 = ax2.scatter(xyz[:,0], xyz[:,1], c=np.log10(_iprop), cmap='PuOr', vmin=-12.75, vmax=-11, s=0.005)
    watershed_workflow.plot.hucs(watershed_workflow.split_hucs.SplitHUCs(my_hucs), crs, ax=ax2, color='navy', lw=0.5)
    ax2.plot(px, py, 'firebrick', lw=1)
    ax2.set_title('Post-fire topsoil\npermeability')
    ax2.set_aspect('equal')
    ax2.axis('off')
    cb = fig.colorbar(p2, fraction=0.04, pad=0.04, extend="both", shrink=0.25, ax=ax2)
    cb.ax.set_title('log K\n[m$^2$]', fontsize=10)
    
    p3 = ax3.scatter(xyz[:,0][ratio==1], xyz[:,1][ratio==1], c='green', s=0.005, label='no-low')
    p3 = ax3.scatter(xyz[:,0][ratio==2], xyz[:,1][ratio==2], c='cyan', s=0.005, label='low')
    p3 = ax3.scatter(xyz[:,0][ratio==3], xyz[:,1][ratio==3], c='yellow', s=0.005, label='moderate')
    p3 = ax3.scatter(xyz[:,0][ratio==4], xyz[:,1][ratio==4], c='red', s=0.005, label='high')
    p3 = ax3.scatter(xyz[:,0][ratio==0], xyz[:,1][ratio==0], c='white', s=0.005, label='_nolegend_')
    watershed_workflow.plot.hucs(watershed_workflow.split_hucs.SplitHUCs(my_hucs), crs, ax=ax3, color='navy', lw=0.5)
    ax3.plot(px, py, 'firebrick', lw=1)
    ax3.set_title('difference')
    ax3.legend(loc='upper right', edgecolor='none', facecolor='white', framealpha=0.5, markerscale=50, fontsize=9)
    ax3.set_aspect('equal')
    ax3.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
np.unique(ratio) if fire else fire

In [ ]:
if fire:
    fig, ax = plt.subplots(1, 1)
    plt.hist(np.log10(iprop), bins=50, alpha=0.5, label='prefire mean='+str(np.around(np.nanmean(np.log10(iprop)),4)))
    plt.hist(np.log10(_iprop), bins=50, alpha=0.5, label='postfire mean='+str(np.around(np.nanmean(np.log10(_iprop)),4)))
    plt.legend()
    plt.show()

In [ ]:
if fire:
    xyzb = np.zeros((len(xyz),4))
    xyzb[:,:3] = xyz
    xyzb[:,-1] = np.log10(_iprop)
    newxyzb = np.copy(xyzb)
    newxyzb[:,2] -= 10
    stacked = np.vstack((xyzb,newxyzb))
    np.savetxt(postfire_perm_4d_file_path, stacked)
    print(stacked.shape)

In [ ]:
# Note the missing data (white).  This is because some SSURGO map units have no formation with complete 
# information.  So we merge the above available data, filling where possible and dropping regions that
# do not have a complete set of properties.
soil_survey_props_clean = soil_survey_props.reset_index()

# later scripts expect 'native_index' as a standard name of holding onto the original IDs
soil_survey_props_clean.rename_axis('native_index', inplace=True)
soil_survey_props_clean.rename(columns={'mukey':'native_index'}, inplace=True)

# need thickness in m
soil_survey_props_clean['thickness [cm]'] = soil_survey_props_clean['thickness [cm]']/100.
soil_survey_props_clean.rename(columns={'thickness [cm]':'thickness [m]'}, inplace=True)


def replace_column_nans(df, col_nan, col_replacement):
    """In a df, replace col_nan entries by col_replacement if is nan.  In Place!"""
    row_indexer = df[col_nan].isna()
    df.loc[row_indexer, col_nan] = df.loc[row_indexer, col_replacement]
    return

# where poro or perm is nan, put Rosetta poro
replace_column_nans(soil_survey_props_clean, 'porosity [-]', 'Rosetta porosity [-]')
replace_column_nans(soil_survey_props_clean, 'permeability [m^2]', 'Rosetta permeability [m^2]')

# drop unnecessary columns
for col in ['Rosetta porosity [-]', 'Rosetta permeability [m^2]', 'bulk density [g/cm^3]', 'total sand pct [%]',
            'total silt pct [%]', 'total clay pct [%]']:
    soil_survey_props_clean.pop(col)
    
# drop nans
soil_survey_props_clean.dropna(inplace=True)
soil_survey_props_clean.reset_index(drop=True, inplace=True)

assert soil_survey_props_clean['porosity [-]'][:].min() >= min_porosity
assert soil_survey_props_clean['permeability [m^2]'][:].max() <= max_permeability
soil_survey_props_clean



In [ ]:
# create a new soil_color, keeping on those that are kept here and re-indexing to ATS indices
soil_color_new = -np.ones_like(soil_color)
for new_id, mukey in enumerate(soil_survey_props_clean['native_index']):
    soil_color_new[np.where(soil_color == mukey)] = 1000+new_id

if generate_plots:
    # image the new soil_color
    indices, cmap, norm, ticks, labels = watershed_workflow.colors.generate_indexed_colormap(soil_color_new, cmap='tab20c')
    fig, ax = watershed_workflow.plot.get_ax(crs)

    mp = watershed_workflow.plot.mesh(m2, crs, ax=ax, facecolor='color',
                        linewidth=0, color=soil_color_new, 
                        cmap=cmap, norm=norm)

    # watershed_workflow.colors.colorbar_index(ncolors=len(np.unique(soil_color_new)), cmap=cmap, labels=labels) 

    ax.set_title('soil type index')
    ax.axis('off')
    fig.savefig(f'../images/{name}/{name}_soil_indices')

### GLYHMPS geologic layer

GLYHMPS is complete in that it does not appear to have missing data, but does not have texture properties needed for Water Retention Models.  Instead we rely on scaling laws to fill the data.

In [ ]:
# extract the GLYHMPS geologic structure data as shapes and project it onto the mesh
target_bounds = watershed.exterior().bounds
logging.info('target bounds: {}'.format(target_bounds))

_, geo_survey, geo_survey_props = watershed_workflow.get_shapes(sources['geologic structure'], 
                                                      target_bounds, crs, crs, properties=True,
                                                      min_porosity=min_porosity, 
                                                      max_permeability=max_permeability,
                                                      max_vg_alpha=max_vg_alpha)

# -- log the bounds targeted and found
# logging.info('shape union bounds: {}'.format(
#     shapely.ops.cascaded_union(geo_survey).bounds))

# -- determine the ID for each soil unit; this uniquely identifies formation
#    properties
geo_ids = np.array([shp.properties['id'] for shp in geo_survey], np.int32)

# -- color a raster by the polygons (this makes identifying a triangle's value much 
#    more efficient)
geo_color_profile, geo_color_raster = \
            watershed_workflow.color_raster_from_shapes(geo_survey, crs, geo_ids,
                                                        target_bounds, 10, crs, -1)

# -- resample the raster to the triangles
geo_color = watershed_workflow.values_from_raster(m2.centroids, crs, 
                                         geo_color_raster, geo_color_profile)


In [ ]:
# select the properties that appear in the mesh
geo_survey_props.set_index('id', inplace=True, drop=False)
geo_survey_props = geo_survey_props.loc[np.unique(geo_color), :]

In [ ]:
if generate_plots:
    # plot the geologic formation id
    indices, cmap, norm, ticks, labels = watershed_workflow.colors.generate_indexed_colormap(geo_color, cmap='tab20b')

    fig, ax = watershed_workflow.plot.get_ax(crs)

    mp = watershed_workflow.plot.mesh(m2, crs, ax=ax, facecolor='color',
                        linewidth=0, color=geo_color, 
                        cmap=cmap, norm=norm)

    # watershed_workflow.colors.colorbar_index(ncolors=len(np.unique(geo_color)), cmap=cmap, labels=labels) 

    ax.set_title('geol type index')
    ax.axis('off')


In [ ]:
if generate_plots:
    # plot permeability of the underlying geologic layer
    iprop = np.empty(geo_color.shape, 'd')
    for i in geo_survey_props.index:
        iprop[geo_color == i] = geo_survey_props.loc[i, 'permeability [m^2]']
    
    fig, ax = watershed_workflow.plot.get_ax(crs)
    mp = watershed_workflow.plot.triangulation(mesh_points3, mesh_tris, crs, ax=ax, 
                                 color=np.log10(iprop), edgecolor='none', cmap='jet')
    cbar = fig.colorbar(mp, shrink=0.5)
    ax.set_title('geology log permeability [m^2]')
    ax.axis('off')

    fig.savefig(f'../images/{name}/{name}_geo_permeability')

In [ ]:
if generate_plots:
    # plot porosity of the geologic layer
    iprop = np.empty(geo_color.shape, 'd')
    for i in geo_survey_props.index:
        iprop[geo_color == i] = geo_survey_props.loc[i, 'porosity [-]']

    fig, ax = watershed_workflow.plot.get_ax(crs)
    mp = watershed_workflow.plot.triangulation(mesh_points3, mesh_tris, crs, ax=ax, 
                                 color=iprop, edgecolor='none', cmap='jet')
    cbar = fig.colorbar(mp, shrink=0.5)
    ax.set_title('geology porosity [-]')
    ax.axis('off')
    fig.savefig(f'../images/{name}/{name}_geo_porosity')

In [ ]:
# note there are clearly some common regions -- no need to duplicate those with identical values.
geo_survey_props_clean = geo_survey_props.copy()
geo_survey_props_clean.pop('logk_stdev [-]')
geo_survey_props_clean.rename(columns={'id':'native_index'}, inplace=True)


def reindex_remove_duplicates(df, index=None):
    """Removes duplicates, creating a new index and saving the old index as tuples of duplicate values. In place!"""
    if index is not None:
        if index in df:
            df.set_index(index, drop=True, inplace=True)
    
    index_name = df.index.name

    # identify duplicate rows
    duplicates = list(df.groupby(list(df)).apply(lambda x: tuple(x.index)))

    # order is preserved
    df.drop_duplicates(inplace=True)
    df.reset_index(inplace=True)
    df[index_name] = duplicates
    return

reindex_remove_duplicates(geo_survey_props_clean, 'native_index')
assert geo_survey_props_clean['porosity [-]'][:].min() >= min_porosity
assert geo_survey_props_clean['permeability [m^2]'][:].max() <= max_permeability
assert geo_survey_props_clean['van Genuchten alpha [Pa^-1]'][:].max() <= max_vg_alpha

geo_survey_props_clean

In [ ]:
# create a new geologic layer color, keeping on those that are kept here and re-indexing to ATS indices
geo_color_new = -np.ones_like(geo_color)
for new_id, old_id_dups in enumerate(geo_survey_props_clean['native_index']):
    for old_id in old_id_dups:
        geo_color_new[np.where(geo_color == old_id)] = 100+new_id
    
if generate_plots:
    # image the new geo_color
    indices, cmap, norm, ticks, labels = watershed_workflow.colors.generate_indexed_colormap(geo_color_new, cmap='tab20c')
    fig, ax = watershed_workflow.plot.get_ax(crs)

    mp = watershed_workflow.plot.mesh(m2, crs, ax=ax, facecolor='color',
                        linewidth=0, color=geo_color_new, 
                        cmap=cmap, norm=norm)

    # watershed_workflow.colors.colorbar_index(ncolors=len(np.unique(geo_color_new)), cmap=cmap, labels=labels) 

    ax.set_title('geologic type index')
    ax.axis('off')
    fig.savefig(f'../images/{name}/{name}_geo_indices')

In [ ]:
assert(geo_color_new.min() > 0)

## Depth-to-bedrock

Depth to bedrock is taken from the [SoilGrids](http://globalchange.bnu.edu.cn/research/dtb.jsp) product.  Here we download a US-based, clipped version of this global product, as file sizes are quite large (all products potentially used total over 100GB).

In [ ]:
# DTB_source = watershed_workflow.source_list.structure_sources['SoilGrids2017']
DTB_profile, DTB_raster = watershed_workflow.get_raster_on_shape(sources['depth to bedrock'], watershed.exterior(), crs, 
                                                       nodata=-99999)#, variable='BDTICM')

# resample the raster to the triangles
DTB_raster = DTB_raster/100 #convert from cm to m
DTB = watershed_workflow.values_from_raster(m2.centroids, crs, DTB_raster, DTB_profile, algorithm='piecewise bilinear')
DTB = np.where(DTB >= 0, DTB, np.nan)

In [ ]:
if generate_plots:
    # plot the resulting surface mesh
    fig, ax = watershed_workflow.plot.get_ax(crs, window=[0.05,0.1,0.9,0.8])
    cbax = fig.add_axes([.95,0.1,0.05,0.8])

    mp = watershed_workflow.plot.triangulation(mesh_points3, mesh_tris, crs, ax=ax, 
                                 color=DTB, cmap='plasma_r', edgecolor='none', linewidth=0.1)
    cbar = fig.colorbar(mp, orientation="vertical", cax=cbax)
    watershed_workflow.plot.hucs(watershed, crs, ax=ax, color='k', linewidth=1)

    ax.set_aspect('equal', 'datalim')
    ax.axis('off')

    cbar.ax.set_title('DTB [m]')
    fig.savefig(f'../images/{name}/{name}_dtb')

## A combined, complete product?

As a default, we would like a material-driven (e.g. not fields for porosity, perm, etc, but soil classes, each with a common porosity/permeability/vG curve) default that is valid everywhere.  That makes it clear that we must rely on GLHYMPS as the only material-based product that is valid everywhere.  Other products may be layered on top of this, replacing GLHYMPS values, but the underlying layer should be based on GLHYMPS.  To fill in the van Genuchten properties, we relate alpha to permeability and choose a single common n and s_r.

Where available, we then choose to use SSURGO as a layer on top of GLHYMPS.  So start by using all GLHYMPS values, then override ones where SSURGO is valid with those values.  This will be the second model, and has then three layers -- a bedrock layer, a soil layer from 0 to 2m, and a geologic layer, using GLHYMPS values.  SoilGrids depth-to-bedrock will be used to provide the transition between bedrock and (where > 2m) the GLHYMPS "geologic" layer or (where < 2m) the SSURGO "soil" layer.  Where SSURGO has no values, the underlying GLHYMPS values will be used even in the top 2m.


Note, all integer IDs in mesh files must be unique.  This includes Material IDs, side sets, etc.  We create the Material ID map and data frame.  This is used to standardize IDs from multiple data sources.  Traditionally, ATS numbers Material IDs/Side Sets as:

* 0-9 : reserved for boundaries, surface/bottom, etc
* 10-99 : Land Cover side sets, typically NLCD IDs are used
* 100-999 : geologic layer material IDs. 999 is reserved for bedrock.
* 1000-9999 : soil layer material IDs




In [ ]:
# map SSURGO mukey to ATS_ID
soil_survey_props_clean['ats_id'] = range(1000, 1000+len(soil_survey_props_clean))
soil_survey_props_clean.set_index('ats_id', inplace=True)

# map GLHYMPS id to ATS_ID
geo_survey_props_clean['ats_id'] = range(100, 100+len(geo_survey_props_clean))
geo_survey_props_clean.set_index('ats_id', inplace=True)

bedrock_props = watershed_workflow.soil_properties.get_bedrock_properties()

# merge the properties databases
subsurface_props = pandas.concat([geo_survey_props_clean,
                                  soil_survey_props_clean,
                                  bedrock_props])

# save the properties to disk for use in generating input file
outputs['subsurface_properties_filename'] = f'../data-processed/{name}/{name}_subsurface_properties.csv'
subsurface_props.to_csv(outputs['subsurface_properties_filename'])
subsurface_props


In [ ]:
# # make a giant plot of all the WRMs
# import plot_wrm # $ATS_SRC_DIR/tools/utils
# fig = plt.figure()
# ax = fig.add_subplot(111)

# cm_gl = watershed_workflow.colors.cm_mapper(100, 103, 'winter')
# cm_ss = watershed_workflow.colors.cm_mapper(1000,1017, 'autumn')

# for i in subsurface_props.index:
#     if i < 999:
#         cl = cm_gl(i)
#     else:
#         cl = cm_ss(i)
#     alpha = subsurface_props.loc[i]['van Genuchten alpha [Pa^-1]']
#     n = subsurface_props.loc[i]['van Genuchten n [-]']
#     sr = subsurface_props.loc[i]['residual saturation [-]']
#     vg = plot_wrm.VanGenuchten(alpha, n, sr)
#     plot_wrm.plot(vg, ax, cl)
    
# # include wilting point limiters
# wp1 = plot_wrm.WiltingPointLimiter(7400, 275000)
# wp2 = plot_wrm.WiltingPointLimiter(6600, 255000)
# wp3 = plot_wrm.WiltingPointLimiter(3500, 224000)
# for wp in [wp1, wp2, wp3]:
#     plot_wrm.plot(wp, ax, 'k')

    
# plt.show()

## Mesh extrusion

Given the surface mesh and material IDs on both the surface and subsurface, we can extrude the surface mesh in the vertical to make a 3D mesh.

The most difficult aspect of extrusion is creating meshes that:
1. aren't huge numbers of cells
2. aren't huge cell thicknesses, especially near the surface
3. follow implied interfaces, e.g. bottom of soil and bottom of geologic layer

This is an iterative process that requires some care and some art.

In [ ]:
# here we choose the bottom of the domain to be the maximum of the depth to bedrock.  
# This is really up to the user, but we are hard-coding this for this watershed_workflow.
dtb_max = np.nanmax(DTB)
DTB = np.where(np.isnan(DTB), dtb_max, DTB)

total_thickness = np.ceil(DTB.max())
print(f'total thickness: {total_thickness} m')

# total_thickness = 41.0

In [ ]:
# Generate a dz structure for the top 2m of soil
#
# here we try for 10 cells, starting at 5cm at the top and going to 50cm at the bottom of the 2m thick soil
dzs, res = watershed_workflow.mesh.optimize_dzs(0.05, 0.5, 2, 8)
print(dzs)
print(sum(dzs))

In [ ]:
# this looks like it would work out, with rounder numbers:
dzs_soil = [0.05, 0.05, 0.05, 0.12, 0.23, 0.5, 0.5, 0.5]
print(sum(dzs_soil))

In [ ]:
# 41m total thickness, minus 2m soil thickness, leaves us with 39 meters to make up.
# optimize again...
dzs2, res2 = watershed_workflow.mesh.optimize_dzs(1, 10, total_thickness-2, 8)
print(dzs2)
print(sum(dzs2))

# how about...
# dzs_geo = [1.0, 3.0, 7.0,] + 3*[12.0,]
# print(dzs_geo)
# print(sum(dzs_geo))

dzs_geo = dzs2.astype(int)
print(dzs_geo)
print(sum(dzs_geo))

In [ ]:
if sum(dzs_geo) != total_thickness - 2:
    dzs_geo[-1] += total_thickness - 2 - sum(dzs_geo)
print(dzs_geo)
print(sum(dzs_geo))

In [ ]:
# layer extrusion
# -- data structures needed for extrusion
layer_types = []
layer_data = []
layer_ncells = []
layer_mat_ids = []

# -- soil layer --
depth = 0
for dz in dzs_soil:
    depth += 0.5 * dz
    layer_types.append('constant')
    layer_data.append(dz)
    layer_ncells.append(1)
    
    if use_geologic_layer:
        # use glhymps params
        br_or_geo = np.where(depth < DTB, geo_color_new, 999)
        soil_or_br_or_geo = np.where(np.bitwise_and(soil_color_new > 0, depth < soil_thickness),
                                 soil_color_new,
                                 br_or_geo)
    else:
        # use ssurgo down to DTB if it exists
        soil_or_geo = np.where(soil_color_new > 0, soil_color_new, geo_color_new)
        soil_or_br_or_geo = np.where(depth < DTB, soil_or_geo, 999)
    layer_mat_ids.append(soil_or_br_or_geo)
    depth += 0.5 * dz
    
# -- geologic layer --
for dz in dzs_geo:
    depth += 0.5 * dz
    layer_types.append('constant')
    layer_data.append(dz)
    layer_ncells.append(1)
    
    if use_geologic_layer:
        geo_or_br = np.where(depth < DTB, geo_color_new, 999)
    else:
        # only soil, no geo
        soil_or_geo = np.where(soil_color_new > 0, soil_color_new, geo_color_new)
        geo_or_br = np.where(depth < DTB, soil_or_geo, 999)
        
    layer_mat_ids.append(geo_or_br)
    depth += 0.5 * dz

# print the summary
watershed_workflow.mesh.Mesh3D.summarize_extrusion(layer_types, layer_data, 
                                            layer_ncells, layer_mat_ids)

# downselect subsurface properties to only those that are used
layer_mat_id_used = list(np.unique(np.array(layer_mat_ids)))
subsurface_props_used = subsurface_props.loc[layer_mat_id_used]
len(subsurface_props_used)


In [ ]:
# extrude
m3 = watershed_workflow.mesh.Mesh3D.extruded_Mesh2D(m2, layer_types, layer_data, 
                                             layer_ncells, layer_mat_ids)

In [ ]:
print('2D labeled sets')
print('---------------')
for ls in m2.labeled_sets:
    print(f'{ls.setid} : {ls.entity} : {len(ls.ent_ids)} : "{ls.name}"')

print('')
print('Extruded 3D labeled sets')
print('------------------------')
for ls in m3.labeled_sets:
    print(f'{ls.setid} : {ls.entity} : {len(ls.ent_ids)} : "{ls.name}"')

print('')
print('Extruded 3D side sets')
print('---------------------')
for ls in m3.side_sets:
    print(f'{ls.setid} : FACE : {len(ls.cell_list)} : "{ls.name}"')
    


In [ ]:
# save to disk
outputs['mesh_filename'] = f'../data-processed/{name}/{name}_{meshsize}-{int(np.round(meshsize*factor))}.exo'
try:
    os.remove(outputs['mesh_filename'])
    os.remove(outputs['mesh_filename'].replace('.exo','.vtk'))
except FileNotFoundError:
    pass
m3.write_exodus(outputs['mesh_filename'])
m3.write_vtk(outputs['mesh_filename'].replace('.exo','.vtk'))

## Collect the DayMet raster covering this area

Note that here we need two files -- the actual data and the typical year data.

The first cell downloads the raw data and generates the actual data file used by ATS, the second cell averages days, smooths the data, and writes a typical year.

In [ ]:
# outputs['daymet_filename'] = f'../data-processed/{name}/{name}_DayMet_1980_2023.h5'

# if generate_daymet:
#     start = "1-1980"
#     end = "365-2023"
#     bounds = watershed.exterior().bounds

#     dat, x, y = watershed_workflow.daymet.collectDaymet(bounds, crs, start, end)
#     ats = watershed_workflow.daymet.daymetToATS(dat)
#     attrs = watershed_workflow.daymet.getAttrs(bounds, start, end)
#     watershed_workflow.daymet.writeHDF5(ats, x, y, attrs, outputs['daymet_filename'])

In [ ]:
outputs['daymet_filename'] = f'../data-processed/{name}/{name}_DayMet_1980_2023.h5'

if generate_daymet:
    startdate = f"{start_year}-1-1"
    enddate = f"{end_year}-12-31"
    bounds = watershed.exterior().bounds
    
    source = watershed_workflow.sources.manager_daymet.FileManagerDaymet()
    data = source.get_data(bounds, crs, startdate, enddate)

    assert(len(data.collections) == 1)
    met_data = data.collections[0]
    met_data_ats = watershed_workflow.daymet.daymet_to_daily_averages(met_data)
    attrs = watershed_workflow.daymet.getAttributes(bounds, startdate, enddate)
    watershed_workflow.io.write_dataset_to_hdf5(outputs['daymet_filename'], met_data_ats.collections[0], attrs)

In [ ]:
# outputs['daymet_spinup_filename'] = f'../data-processed/{name}/{name}_DayMet_typical_1980_2023.h5'

# if generate_daymet:
#     ats_typ = watershed_workflow.daymet.daymetToATS(dat, smooth=True, smooth_filter=True, nyears=43)
#     watershed_workflow.daymet.writeHDF5(ats_typ, x, y, attrs, outputs['daymet_spinup_filename'])

In [ ]:
outputs['daymet_spinup_filename'] = f'../data-processed/{name}/{name}_DayMet_typical_1980_2023.h5'

if generate_daymet:
    data_typ = source.get_data(bounds, crs, startdate, enddate)
    met_data_typ = data_typ.collections[0]
    
    logging.info("averaging daymet by taking the average for each day across the actual years.")
    window, poly_order = 61, 2
    for key in data.collections[0]:
        logging.info(f"smoothing {key} using savgol filter, window = {window} d, poly order = {poly_order}")
        met_data_typ.data[key] = watershed_workflow.utils.compute_average_year(met_data_typ.data[key], output_nyears=end_year-start_year+1, filter=True)    
    
    met_data_typ_ats = watershed_workflow.daymet.daymet_to_daily_averages(met_data_typ)
    watershed_workflow.io.write_dataset_to_hdf5(outputs['daymet_spinup_filename'], met_data_typ_ats.collections[0], attrs)

## Generate files for Leaf Area Index

In addition to meteorological forcing data, we need land cover leaf area index data.  Typically this can be smoother than the DayMet, and our default source for this, MODIS, is not currently automated.  Therefore this assumes you have already acquired the raw MODIS data (note -- get Pin to add his scripts for this!)

MODIS is available from 2002 to present -- we generate a typical year and can use that for all years, though probably we should do something smarter if possible.

In [ ]:
def plot(df, form, axs):
    cm = watershed_workflow.colors.enumerated_colors(3)
    i = 0
    for k in df.keys():
        if not k.startswith('time'):
            axs[i].plot(df['time [d]'], df[k], form, color=cm[i])
            axs[i].set_title(k)
            i += 1

In [ ]:
# load the raw MODIS data and covert it to pandas
ats_raw_modis_filename = f'../data-processed/{name}/{name}_MODIS_LAI_07042002_02102024.h5'

if generate_modis:
    d = h5py.File(ats_raw_modis_filename,'r')
    df = pandas.DataFrame()
    for k in d.keys():
        df[k] = d[k][:]
    df['time [d]'] = df['time [s]']/86400
df if generate_modis else generate_modis

In [ ]:
if generate_modis:
    # interpolate this time series into a daily time series
    # ts = np.arange(8214, 14600, 1)
    ts = np.arange(df['time [d]'].values[-1]+1)
    df_interp = pandas.DataFrame()
    df_interp['time [d]'] = ts

    for k in df.keys():
        if k != 'time [s]':
            f = scipy.interpolate.interp1d(df['time [d]'][:], df[k][:])
            df_interp[k] = f(ts)

    df = df_interp
    df['datetime'] = pandas.to_datetime(df['time [d]'], unit='D', origin=pandas.Timestamp('2002-10-01'))
df if generate_modis else generate_modis

In [ ]:
if generate_modis:
    leap_index = []
    for i in range(len(df)):
        if '02-29' in str(df.iloc[i,-1]):
            leap_index.append(i)
            print(i, df.iloc[i,-1])

In [ ]:
if generate_modis:
    df.drop(leap_index, inplace=True)
    df.reset_index(drop=True, inplace=True)
    df['time [d]'] = np.arange(len(df))
df if generate_modis else generate_modis

In [ ]:
if generate_modis:
    for i in range(len(df)):
        if '02-29' in str(df.iloc[i,-1]):
            print(i, df.iloc[i,-1])

In [ ]:
if generate_modis:
    df.drop(columns=['datetime'], inplace=True)
df if generate_modis else generate_modis

In [ ]:
if generate_modis:
    # smooth the data
    df_smooth = pandas.DataFrame()
    df_smooth['time [d]'] = df['time [d]']
    for k in df.keys():
        if k != 'time [d]':
            df_smooth[k] = scipy.signal.savgol_filter(df[k], 101, 3)

    if generate_plots:
        df_smooth.iloc[:,1:].plot()
        # # plot comparison
        # fig = plt.figure()
        # axs = fig.subplots(3,1)
        # # plot(df, '-', axs)
        # plot(df_smooth, '-', axs)
        # plt.tight_layout()
        plt.show()
        
        
df_smooth if generate_modis else generate_modis

In [ ]:
# add time back and write to disk
outputs['modis_filename'] = f'../data-processed/{name}/{name}_MODIS_LAI_07042002_02102024_smoothed.h5'

if generate_modis:
    df_smooth['time [s]'] = df_smooth['time [d]']*86400
    with h5py.File(outputs['modis_filename'],'w') as fid:
        for k in df_smooth:
            fid.create_dataset(k, data=df_smooth[k][:])

In [ ]:
if generate_modis:
    df = df_smooth
    # split into n_years dataframes, one per year
    df_yr = []
    for year in range(20):
        yr = df.loc[df_interp['time [d]'] >= year*365]
        df_yr.append(yr.loc[yr['time [d]'] < (year+1)*365])

    # average across the years
    df_avg = pandas.DataFrame()
    for yr in df_yr:
        for k in yr.keys():
            if not k.startswith('time'):
                if k in df_avg:
                    df_avg[k] = df_avg[k].array + yr[k].array
                else:
                    df_avg[k] = yr[k].copy()

    for k in df_avg.keys():
        df_avg[k] = df_avg[k][:] / len(df_yr)

    df_avg['time [d]'] = df['time [d]']
    
df_avg if generate_modis else generate_modis

In [ ]:
if generate_modis:    
    if generate_plots:
        df_avg.iloc[:,:-1].plot()
        # fig = plt.figure()
        # axs = fig.subplots(3,1)
        # plot(df_avg, '-', axs)
        # plt.tight_layout()
        plt.show()

In [ ]:
nyears = end_year-start_year+1 # to match DayMet 1980-2023
if generate_modis:
    # replicate nyears times to make nyears years (remem)
    # tile all data to repeat n_year times
    df_repeat = pandas.DataFrame()
    for key in df_avg:
        if not key.startswith('time'):
            df_repeat[key] = np.tile(df_avg[key].array, nyears)
            assert(len(df_repeat) == nyears*365)
    
    # time is simply daily data
    df_repeat['time [d]'] = np.arange(0., nyears * 365., 1.)
    df_repeat['time [s]'] = 86400*df_repeat['time [d]']
    
df_repeat if generate_modis else generate_modis

In [ ]:
if generate_modis and generate_plots:
    df_repeat.iloc[:,:-2].plot()
    # plot this and make sure it looks right
    # fig = plt.figure()
    # axs = fig.subplots(3,1)
    # plot(df_repeat, '-', axs)
    # plt.tight_layout()
    plt.show()

In [ ]:
# write to disk
outputs['modis_typical_filename'] = f'../data-processed/{name}/{name}_MODIS_LAI_07042002_02102024_typical{nyears}yr.h5'

if generate_modis:
    with h5py.File(outputs['modis_typical_filename'],'w') as fid:
        for k in df_repeat:
            fid.create_dataset(k, data=df_repeat[k][:])

In [ ]:
# add 274 to typical
if generate_modis:
    d = h5py.File(outputs['modis_typical_filename'],'r')
    df = pandas.DataFrame()
    for k in d.keys():
        if k != 'time [d]':
            df[k] = d[k][:]
    d.close()
    df['time [s]'] += 274*86400
    with h5py.File(outputs['modis_typical_filename'],'w') as fid:
        for k in df:
            fid.create_dataset(k, data=df[k][:])
df if generate_modis else generate_modis

In [ ]:
# add 274 to smoothed
if generate_modis:
    d = h5py.File(outputs['modis_filename'],'r')
    df = pandas.DataFrame()
    for k in d.keys():
        if k != 'time [d]':
            df[k] = d[k][:]
    d.close()
    df['time [s]'] += (274+365*22)*86400
    with h5py.File(outputs['modis_filename'],'w') as fid:
        for k in df:
            fid.create_dataset(k, data=df[k][:])
df if generate_modis else generate_modis

## Write ATS input files

We now generate three input files -- two for spinup (steadystate solution and cyclic steadystate solution) and one for transient runs.

Steadystate has its own physics, but cyclic steadystate and transient share a common set of physics.  Each have their own met data strategy.

The first step is to generate the sections of xml that will replace parts of the template files.  This is done prior to loading any templates to make clear that these are totally generated from scratch using the ats_input_spec tool.

Note that throughout, we will assume an additional level of folder nesting, e.g. runs will be completed in '../spinup-CoalCreek/run0', meaning that we have to append an extra '../' to the start of all filenames.  This makes it easier to deal with mistakes, continued runs, etc.

In [ ]:
# add the subsurface and surface domains
#
# Note this also adds a "computational domain" region to the region list, and a vis spec 
# for "domain"
def add_domains(main_list, mesh_filename, surface_region='surface', snow=True, canopy=True):
    ats_input_spec.public.add_domain(main_list, 
                                 domain_name='domain', 
                                 dimension=3, 
                                 mesh_type='read mesh file',
                                 mesh_args={'file':mesh_filename})
    if surface_region:
        main_list['mesh']['domain']['build columns from set'] = surface_region    
    
        # Note this also adds a "surface domain" region to the region list and a vis spec for 
        # "surface"
        ats_input_spec.public.add_domain(main_list,
                                domain_name='surface',
                                dimension=2,
                                mesh_type='surface',
                                mesh_args={'surface sideset name':'surface'})
    if snow:
        # Add the snow and canopy domains, which are aliases to the surface
        ats_input_spec.public.add_domain(main_list,
                                domain_name='snow',
                                dimension=2,
                                mesh_type='aliased',
                                mesh_args={'target':'surface'})
    if canopy:
        ats_input_spec.public.add_domain(main_list,
                                domain_name='canopy',
                                dimension=2,
                                mesh_type='aliased',
                                mesh_args={'target':'surface'})


In [ ]:
def add_land_cover(main_list):
    # next write a land-cover section for each NLCD type
    for index, nlcd_name in zip(nlcd_indices, nlcd_labels):
        ats_input_spec.public.set_land_cover_default_constants(main_list, nlcd_name)

    land_cover_list = main_list['state']['initial conditions']['land cover types']
    # update some defaults
    # ['Other', 'Deciduous Forest', 'Evergreen Forest', 'Shrub/Scrub']
    # note, these are from the CLM Technical Note v4.5
    #
    # Rooting depth curves from CLM TN 4.5 table 8.3
    #
    # Note, the mafic potential values are likely pretty bad for the types of van Genuchten 
    # curves we are using (ETC -- add paper citation about this topic).  Likely they need
    # to be modified.  Note that these values are in [mm] from CLM TN 4.5 table 8.1, so the 
    # factor of 10 converts to [Pa]
    #
    # Note, albedo of canopy taken from CLM TN 4.5 table 3.1
    if 42 in nlcd_color_new:
        land_cover_list['Evergreen Forest']['rooting profile alpha [-]'] = 7.0
        land_cover_list['Evergreen Forest']['rooting profile beta [-]'] = 2.0
        land_cover_list['Evergreen Forest']['rooting depth max [m]'] = 10.0
        land_cover_list['Evergreen Forest']['mafic potential at fully closed stomata [Pa]'] = 255000
        land_cover_list['Evergreen Forest']['mafic potential at fully open stomata [Pa]'] = 66000 * .1
        land_cover_list['Evergreen Forest']['albedo of canopy [-]'] = 0.07

    if 41 in nlcd_color_new:
        land_cover_list['Deciduous Forest']['rooting profile alpha [-]'] = 6.0
        land_cover_list['Deciduous Forest']['rooting profile beta [-]'] = 2.0
        land_cover_list['Deciduous Forest']['rooting depth max [m]'] = 10.0
        land_cover_list['Deciduous Forest']['mafic potential at fully closed stomata [Pa]'] = 224000
        land_cover_list['Deciduous Forest']['mafic potential at fully open stomata [Pa]'] = 35000 * .10
        land_cover_list['Deciduous Forest']['albedo of canopy [-]'] = 0.1



In [ ]:

# add soil sets: note we need a way to name the set, so we use, e.g. SSURGO-MUKEY.
def soil_set_name(ats_id):
    if ats_id == 999:
        return 'bedrock'
    source = subsurface_props_used.loc[ats_id]['source']
    native_id = subsurface_props_used.loc[ats_id]['native_index']
    if type(native_id) in [tuple,list]:
        native_id = native_id[0]
    return f"{source}-{native_id}"


# get an ATS "main" input spec list -- note, this is a dummy and is not used to write any files yet
def get_main():
    main_list = ats_input_spec.public.get_main()
    flow_pk = ats_input_spec.public.add_leaf_pk(main_list, 'flow', main_list['cycle driver']['PK tree'], 
                                            'richards-spec')

    # add the mesh and all domains
    mesh_filename = os.path.join('.', outputs['mesh_filename'])
    add_domains(main_list, mesh_filename)

    # add labeled sets
    for ls in m3.labeled_sets:
        ats_input_spec.public.add_region_labeled_set(main_list, ls.name, ls.setid, mesh_filename, ls.entity)
    for ss in m3.side_sets:
        ats_input_spec.public.add_region_labeled_set(main_list, ss.name, ss.setid, mesh_filename, 'FACE')
    
    # add land cover
    add_land_cover(main_list)

    # add soil material ID regions, porosity, permeability, and WRMs
    for ats_id in subsurface_props_used.index:
        props = subsurface_props_used.loc[ats_id]
        set_name = soil_set_name(ats_id)
        
        if props['van Genuchten n [-]'] < 1.5:
            smoothing_interval = 0.01
        else:
            smoothing_interval = 0.0
        
        ats_input_spec.public.add_soil_type(main_list, set_name, ats_id, mesh_filename,
                                            float(props['porosity [-]']),
                                            float(props['permeability [m^2]']), 1.e-7,
                                            float(props['van Genuchten alpha [Pa^-1]']),
                                            float(props['van Genuchten n [-]']),
                                            float(props['residual saturation [-]']),
                                            float(smoothing_interval))
    # print(main_list)  
    # add observations for each subcatchment for transient runs
    ats_input_spec.public.add_observations_water_balance(main_list, "computational domain", 
                                                         "surface domain","external_sides")
    for poly in watershed.polygons():
        region = poly.properties[huc_key]
        ats_input_spec.public.add_observations_water_balance(main_list, region, outlet_region=region+' outlet')
    
    
    return main_list

# create the main list
main_list = get_main()

outputs['generated_ats'] = f'../data-processed/{name}/{name}_generated_ats.xml'
ats_input_spec.io.write(main_list, outputs['generated_ats'])
main_xml = ats_input_spec.io.to_xml(main_list)

In [ ]:
def populate_basic_properties(xml, main_xml, homogeneous_wrm=False, homogeneous_poro=False, homogeneous_perm=False):
    """This function updates an xml object with the above properties for mesh, regions, soil props, and lc props"""
    # find and replace the mesh list
    mesh_i = next(i for (i,el) in enumerate(xml) if el.get('name') == 'mesh')
    xml[mesh_i] = asearch.child_by_name(main_xml, 'mesh')

    # find and replace the regions list
    region_i = next(i for (i,el) in enumerate(xml) if el.get('name') == 'regions')
    xml[region_i] = asearch.child_by_name(main_xml, 'regions')

    # find and replace the WRMs list -- note here we only replace the inner "WRM parameters" because the
    # demo has this in the PK, not in the evaluators list
    if not homogeneous_wrm:
        wrm_list = asearch.find_path(xml, ['PKs', 'water retention evaluator'])
        wrm_i = next(i for (i,el) in enumerate(wrm_list) if el.get('name') == 'WRM parameters')
        wrm_list[wrm_i] = asearch.find_path(main_xml, ['PKs','water retention evaluator','WRM parameters'])

    fe_list = asearch.find_path(xml, ['state', 'evaluators'])

    # find and replace porosity, permeability
    if not homogeneous_poro:
        poro_i = next(i for (i,el) in enumerate(fe_list) if el.get('name') == 'base_porosity')
        fe_list[poro_i] = asearch.find_path(main_xml, ['state', 'evaluators', 'base_porosity'])

    if not homogeneous_perm:
        perm_i = next(i for (i,el) in enumerate(fe_list) if el.get('name') == 'permeability')
        fe_list[perm_i] = asearch.find_path(main_xml, ['state', 'evaluators', 'permeability'])

    # find and replace land cover
    consts_list = asearch.find_path(xml, ['state', 'initial conditions'])
    try:
        lc_i = next(i for (i,el) in enumerate(consts_list) if el.get('name') == 'land cover types')
    except StopIteration:
        pass
    else:
        consts_list[lc_i] = asearch.find_path(main_xml, ['state', 'initial conditions', 'land cover types'])

def create_unique_name(name, homogeneous_wrm=False, homogeneous_poro=False, homogeneous_perm=False):
    suffix = '_h'
    if homogeneous_perm:
        suffix += 'K'
    if homogeneous_poro:
        suffix += 'p'
    if homogeneous_wrm:
        suffix += 'w'
    if suffix == '_h':
        suffix = ''
    return name + suffix
        

For the first file, we load a spinup template and write the needed quantities into that file, saving it to the appropriate run directory.  Note there is no DayMet or land cover or LAI properties needed for this run.  The only property that is needed is the domain-averaged, mean annual rainfall rate.  We then take off some for ET (note too wet spins up faster than too dry, so don't take off too much...).

In [ ]:
if generate_daymet:
    # calculate the basin-averaged, annual-averaged precip rate
    precip_total = ats_typ['precipitation rain [m s^-1]'] + ats_typ['precipitation snow [m SWE s^-1]']
    mean_precip = precip_total.mean()
else:
    rain = 'precipitation rain [m s^-1]'
    snow = 'precipitation snow [m SWE s^-1]'
    try:
        with h5py.File(outputs['daymet_spinup_filename'], 'r') as fid:
            mean_precip = np.array([fid[rain][ts][:].mean() + fid[snow][ts][:].mean() for ts in fid[rain].keys()]).mean()
    except:
        mean_precip = 5e-8
        
logging.info(f'Mean annual precip rate [m s^-1] = {mean_precip}')

In [ ]:
def write_spinup_steadystate(name, mean_precip, **kwargs):
    name = create_unique_name(name, **kwargs)
    logging.info(f'Writing transient: {name}')
    
    # write the spinup xml file
    # load the template file
    xml = aio.fromFile('spinup_steadystate-template.xml')

    # populate basic properties for mesh, regions, and soil properties
    populate_basic_properties(xml, main_xml, **kwargs)

    # set the mean avg source as 60% of mean precip
    precip_el = asearch.find_path(xml, ['state', 'evaluators', 'surface-precipitation', 
                                        'function-constant', 'value'])
    precip_el.setValue(mean_precip * .6)

    # write to disk
    outputs[f'spinup_steadystate_{name}_filename'] = f'../run0-spinup_steadystate/{name}_{meshsize}-{int(np.round(meshsize*factor))}.xml'
    aio.toFile(xml, outputs[f'spinup_steadystate_{name}_filename'])

    # make a run directory
    outputs[f'spinup_steadystate_{name}_rundir'] = f'../run0-spinup_steadystate/{name}-0'
    try:
        os.mkdir(outputs[f'spinup_steadystate_{name}_rundir'])
    except FileExistsError:
        pass

For the second file, we load a transient run template.  This file needs the basics, plus DayMet and LAI as the "typical year data".  Also we set the run directory that will be used for the steadystate run.

For the third file, we load a transient run template as well.  This file needs the basics, DayMet with the actual data, and we choose for this run to use the MODIS typical year.  MODIS is only available for 2002 on, so if we didn't need 1980-2002 we could use the real data, but for this run we want a longer record.

In [ ]:
def write_transient(name, cyclic_steadystate=False, start_year=1980, end_year=2023, **kwargs):
    # make a unique name based on options
    name = create_unique_name(name, **kwargs)
    logging.info(f'Writing transient: {name}')

    if cyclic_steadystate:
        prefix = 'spinup_cyclic'
        # start_year = 1980
        # end_year = 2023
        previous = 'spinup_steadystate'
        runnum = 'run1'
    else:
        prefix = 'transient'
        previous = 'spinup_cyclic'
        runnum = 'run2'

    template_filename = f'{prefix}-template.xml'
    
    # write the cyclic spinup xml file
    # load the template file
    xml = aio.fromFile(template_filename)

    # populate basic properties for mesh, regions, and soil properties
    populate_basic_properties(xml, main_xml, **kwargs)

    # update the DayMet filenames
    if cyclic_steadystate:
        daymet_filename = outputs['daymet_spinup_filename']
    else:
        daymet_filename = outputs['daymet_filename']
    for var in ['surface-incoming_shortwave_radiation',
                'surface-precipitation_rain',
                'snow-precipitation',
                'surface-air_temperature',
                'surface-vapor_pressure_air',
                'surface-temperature',
                'canopy-temperature']:
        try:
            par = asearch.find_path(xml, ['state', 'evaluators', var, 'file'])
        except aerrors.MissingXMLError:
            pass
        else:
            par.setValue(os.path.join('.', daymet_filename))

    # update the LAI filenames
    for par in asearch.findall_path(xml, ['canopy-leaf_area_index', 'file']):
        if cyclic_steadystate:
            par.setValue(os.path.join('.', outputs['modis_typical_filename']))
        else:
            par.setValue(os.path.join('.', outputs['modis_filename']))
    
    # update the start and end time -- start at Oct 1 of year 0, end 10 years later
    start_day = 274 + 365*(start_year - 1980)
    par = asearch.find_path(xml, ['cycle driver', 'start time'])
    par.setValue(start_day)

    end_day = 274 + 365*(end_year - 1980)
    par = asearch.find_path(xml, ['cycle driver', 'end time'])
    par.setValue(end_day)
    
    # update the restart filenames
    for var in asearch.findall_path(xml, ['initial condition', 'restart file']):
        var.setValue(os.path.join('.', outputs[f'{previous}_{name}_rundir'], 'checkpoint_final.h5'))

    # update the observations list
    obs = next(i for (i,el) in enumerate(xml) if el.get('name') == 'observations')
    xml[obs] = asearch.child_by_name(main_xml, 'observations')
   
        
    # write to disk and make a directory for running the run
    outputs[f'{prefix}_{name}_filename'] = f'../{runnum}-{prefix}/{name}_{meshsize}-{int(np.round(meshsize*factor))}.xml'
    filename = outputs[f'{prefix}_{name}_filename']

    outputs[f'{prefix}_{name}_rundir'] = f'../{runnum}-{prefix}/{name}-0'
    rundir = outputs[f'{prefix}_{name}_rundir']

    aio.toFile(xml, filename)
    try:
        os.mkdir(rundir)
    except FileExistsError:
        pass


In [ ]:
# create the fully-heterogeneous runs
if include_heterogeneous:
    write_spinup_steadystate(name, mean_precip)
    write_transient(name, True)
    write_transient(name, False)

# create homogeneous runs
if include_homogeneous:
    write_spinup_steadystate(name, mean_precip, homogeneous_wrm=True, homogeneous_poro=True, homogeneous_perm=True)
    write_transient(name, True, homogeneous_wrm=True, homogeneous_poro=True, homogeneous_perm=True)
    write_transient(name, False, homogeneous_wrm=True, homogeneous_poro=True, homogeneous_perm=True)
    
if include_homogeneous_wrm:
    write_spinup_steadystate(name, mean_precip, homogeneous_wrm=True, homogeneous_poro=False, homogeneous_perm=False)
    write_transient(name, True, homogeneous_wrm=True, homogeneous_poro=False, homogeneous_perm=False)
    write_transient(name, False, homogeneous_wrm=True, homogeneous_poro=False, homogeneous_perm=False)
    
if include_homogeneous_wrm_porosity:
    write_spinup_steadystate(name, mean_precip, homogeneous_wrm=True, homogeneous_poro=True, homogeneous_perm=False)
    write_transient(name, True, homogeneous_wrm=True, homogeneous_poro=True, homogeneous_perm=False)
    write_transient(name, False, homogeneous_wrm=True, homogeneous_poro=True, homogeneous_perm=False)
    
if include_homogeneous_wrm_permeability:
    write_spinup_steadystate(name, mean_precip, homogeneous_wrm=True, homogeneous_poro=False, homogeneous_perm=True)
    write_transient(name, True, homogeneous_wrm=True, homogeneous_poro=False, homogeneous_perm=True)
    write_transient(name, False, homogeneous_wrm=True, homogeneous_poro=False, homogeneous_perm=True)


In [ ]:
logging.info('this workflow is a total success')